# Clasificacion Baggins
Para probar el clasificador con baggins vamos a utilizar las dos técnicas de selección de características y de reducción de instancias que mejor nos han funcionado en el apartado anterior:
- mrMr con centroides soft
- RFC con centroides soft

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report

In [2]:
import pandas as pd

data_path = './data/'


train_data = pd.read_csv(f'{data_path}train_data.csv')
test_data = pd.read_csv(f'{data_path}test_data.csv')

# Selección de características con mrMr
df_reduce_mrmr = pd.read_csv(f'{data_path}X_train_reduce_mrmr.csv')
df_reduce_mrmr_instances = pd.read_csv(f'{data_path}df_reduce_mrmr_instances.csv')
df_reduce_mrmr_instances_hard = pd.read_csv(f'{data_path}df_reduce_mrmr_instances_hard.csv')
df_reduce_mrmr_instances_GLVQ = pd.read_csv(f'{data_path}df_reduce_mrmr_instances_GLVQ.csv')

# Selección de características con RFC
df_X_train_reduce_RFC = pd.read_csv(f'{data_path}df_X_train_reduce_RFC.csv')
df_reduce_RFC_instances = pd.read_csv(f'{data_path}df_reduce_RFC_instances.csv')
df_reduce_RFC_instances_hard = pd.read_csv(f'{data_path}df_reduce_RFC_instances_hard.csv')
df_reduce_RFC_instances_GLVQ = pd.read_csv(f'{data_path}df_reduce_RFC_instances_GLVQ.csv')

# Mostrar información sobre los DataFrames cargados
print("Datos cargados exitosamente:")
print(f"train_data: {train_data.shape}")
print(f"df_reduce_mrmr: {df_reduce_mrmr.shape}")
print(f"df_reduce_mrmr_instances: {df_reduce_mrmr_instances.shape}")
print(f"df_reduce_mrmr_instances_GLVQ: {df_reduce_mrmr_instances_GLVQ.shape}")
print(f"df_X_train_reduce_RFC: {df_X_train_reduce_RFC.shape}")
print(f"df_reduce_RFC_instances: {df_reduce_RFC_instances.shape}")
print(f"df_reduce_RFC_instances_GLVQ: {df_reduce_RFC_instances_GLVQ.shape}")

Datos cargados exitosamente:
train_data: (256326, 31)
df_reduce_mrmr: (256326, 11)
df_reduce_mrmr_instances: (886, 11)
df_reduce_mrmr_instances_GLVQ: (2, 11)
df_X_train_reduce_RFC: (256326, 11)
df_reduce_RFC_instances: (886, 11)
df_reduce_RFC_instances_GLVQ: (2, 11)


### Baggins con mrMr clusterCentroids_soft

#### DecisionTreeClassifier

In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = DecisionTreeClassifier()
bagging = BaggingClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 20, 50, 100],  
    'max_samples': [0.5, 0.7, 1.0],    
    'max_features': [0.5, 0.7, 1.0],   
    'bootstrap': [True, False],        
    'bootstrap_features': [True, False] 
}

# Configura el GridSearchCV
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evalúa el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))


Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)


KeyboardInterrupt: 

## COMPROBAMOS EL RESULTADO CON EL CONJUNTO DE TEST

In [11]:
from sklearn.preprocessing import MinMaxScaler
# Cogemos los datos de test y les eliminamos las cracterísticas que no necesitamos
X = test_data.drop(columns=['Class'])
y_test_final = test_data['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']
X_reduce = X[columns_to_keep_mrmr]

#normalizamos la entrada
scaler = MinMaxScaler()
# Normalizar las columnas
X_reduce['Amount'] = scaler.fit_transform(X_reduce[['Amount']])
X_reduce['Time'] = scaler.fit_transform(X_reduce[['Time']])

print(y_test_final.value_counts())

Class
0    28432
1       49
Name: count, dtype: int64


/tmp/ipykernel_63839/3588149006.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_reduce['Amount'] = scaler.fit_transform(X_reduce[['Amount']])
/tmp/ipykernel_63839/3588149006.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_reduce['Time'] = scaler.fit_transform(X_reduce[['Time']])


In [15]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Configura el modelo con los mejores parámetros
bagging_best = BaggingClassifier(
    estimator=DecisionTreeClassifier(),  
    n_estimators=50,                     
    max_samples=0.7,                     
    max_features=1.0,                   
    bootstrap=False,
    bootstrap_features=True,
    random_state=0
)

X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Entrena el modelo con los datos de entrenamiento
bagging_best.fit(X_train, y_train)

# Evalúa el modelo en los datos de prueba
y_pred_dtc = bagging_best.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_dtc)
report = classification_report(y_test_final, y_pred_dtc, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)



Matriz de confusión:
[[14303 14129]
 [    2    47]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.50      0.67     28432
Fraudulentas       0.00      0.96      0.01        49

    accuracy                           0.50     28481
   macro avg       0.50      0.73      0.34     28481
weighted avg       1.00      0.50      0.67     28481



#### SVC

In [ ]:

from sklearn.svm import SVC
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = SVC(probability=True)
bagging = BaggingClassifier(estimator=base_estimator, random_state=0)

# Configura el GridSearchCV
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evalúa el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))


Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.5, 'max_samples': 1.0, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.8757067226051344
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.79      0.98      0.87        89
           1       0.97      0.74      0.84        89

    accuracy                           0.86       178
   macro avg       0.88      0.86      0.86       178
weighted avg       0.88      0.86      0.86       178



## COMPROBAMOS EL RESULTADO CON EL CONJUNTO DE TEST

In [ ]:
from sklearn.ensemble import BaggingClassifier

# Configura el modelo con los mejores parámetros
bagging_best = BaggingClassifier(
    estimator=SVC(probability=True),  
    n_estimators=10,                     
    max_samples=1.0,                     
    max_features=0.5,                   
    bootstrap=False,
    bootstrap_features=True,
    random_state=0
)

X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Entrena el modelo con los datos de entrenamiento
bagging_best.fit(X_train, y_train)

# Evalúa el modelo en los datos de prueba
y_pred_svc = bagging_best.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_svc)
report = classification_report(y_test_final, y_pred_svc, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)



Matriz de confusión:
[[28282   150]
 [    7    42]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      1.00     28432
Fraudulentas       0.22      0.86      0.35        49

    accuracy                           0.99     28481
   macro avg       0.61      0.93      0.67     28481
weighted avg       1.00      0.99      1.00     28481



#### GaussianNB

In [18]:

from sklearn.naive_bayes import GaussianNB
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = GaussianNB()
bagging = BaggingClassifier(estimator=base_estimator, random_state=0)

# Configura el GridSearchCV
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evalúa el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))


Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.895435021476376
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.88      0.75      0.81        89
           1       0.78      0.90      0.84        89

    accuracy                           0.83       178
   macro avg       0.83      0.83      0.82       178
weighted avg       0.83      0.83      0.82       178



## COMPROBAMOS EL RESULTADO CON EL CONJUNTO DE TEST

In [21]:

# Configura el modelo con los mejores parámetros
bagging_best = BaggingClassifier(
    estimator=GaussianNB(),  
    n_estimators=10,                     
    max_samples=0.5,                     
    max_features=0.5,                   
    bootstrap=True,
    bootstrap_features=True,
    random_state=0
)

X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Entrena el modelo con los datos de entrenamiento
bagging_best.fit(X_train, y_train)

# Evalúa el modelo en los datos de prueba
y_pred_gnb = bagging_best.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_gnb)
report = classification_report(y_test_final, y_pred_gnb, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[17420 11012]
 [    3    46]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.61      0.76     28432
Fraudulentas       0.00      0.94      0.01        49

    accuracy                           0.61     28481
   macro avg       0.50      0.78      0.38     28481
weighted avg       1.00      0.61      0.76     28481



#### RandomForestClassifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = RandomForestClassifier()
bagging = BaggingClassifier(estimator=base_estimator, random_state=0)

# Configura el GridSearchCV
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evalúa el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))


Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.7, 'max_samples': 1.0, 'n_estimators': 100}
Mejor puntuación de validación cruzada: 0.963270402557187
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.98      0.99      0.98        89
           1       0.99      0.98      0.98        89

    accuracy                           0.98       178
   macro avg       0.98      0.98      0.98       178
weighted avg       0.98      0.98      0.98       178



## COMPROBAMOS EL RESULTADO CON EL CONJUNTO DE TEST

In [22]:

# Configura el modelo con los mejores parámetros
bagging_best = BaggingClassifier(
    estimator=RandomForestClassifier(),  
    n_estimators=100,                     
    max_samples=1.0,                     
    max_features=0.7,                   
    bootstrap=False,
    bootstrap_features=True,
    random_state=0
)

X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Entrena el modelo con los datos de entrenamiento
bagging_best.fit(X_train, y_train)

# Evalúa el modelo en los datos de prueba
y_pred_rfc = bagging_best.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_rfc)
report = classification_report(y_test_final, y_pred_rfc, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[15026 13406]
 [    2    47]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.53      0.69     28432
Fraudulentas       0.00      0.96      0.01        49

    accuracy                           0.53     28481
   macro avg       0.50      0.74      0.35     28481
weighted avg       1.00      0.53      0.69     28481



### Baggins con RFC clusterCentroids_soft

#### DecisionTreeClassifier

In [23]:
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = DecisionTreeClassifier()
bagging = BaggingClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 20, 50, 100],  
    'max_samples': [0.5, 0.7, 1.0],    
    'max_features': [0.5, 0.7, 1.0],   
    'bootstrap': [True, False],        
    'bootstrap_features': [True, False] 
}

# Configura el GridSearchCV
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evalúa el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))


Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 100}
Mejor puntuación de validación cruzada: 0.9265507941264609
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.88      0.96      0.91        89
           1       0.95      0.87      0.91        89

    accuracy                           0.91       178
   macro avg       0.91      0.91      0.91       178
weighted avg       0.91      0.91      0.91       178



## COMPROBAMOS EL RESULTADO CON EL CONJUNTO DE TEST

In [3]:
X = test_data.drop(columns=['Class'])
y_test_final = test_data['Class']
### Version con RandomForestClassifier
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']
X_reduce_RFC = X[columns_to_keep_RFC]

In [9]:

# Configura el modelo con los mejores parámetros
bagging_best = BaggingClassifier(
    estimator=DecisionTreeClassifier(),  
    n_estimators=10,                     
    max_samples=0.5,                     
    max_features=0.5,                   
    bootstrap=True,
    bootstrap_features=True,
    random_state=0
)

X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Entrena el modelo con los datos de entrenamiento
bagging_best.fit(X_train, y_train)

# Evalúa el modelo en los datos de prueba
y_pred_rfc = bagging_best.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_rfc)
report = classification_report(y_test_final, y_pred_rfc, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[27911   521]
 [    5    44]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.98      0.99     28432
Fraudulentas       0.08      0.90      0.14        49

    accuracy                           0.98     28481
   macro avg       0.54      0.94      0.57     28481
weighted avg       1.00      0.98      0.99     28481



### SVC

In [10]:

from sklearn.svm import SVC
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = SVC(probability=True)
bagging = BaggingClassifier(estimator=base_estimator, random_state=0)

# Configura el GridSearchCV
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evalúa el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))


Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'bootstrap': True, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.9209169913095595
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.86      0.97      0.91        89
           1       0.96      0.84      0.90        89

    accuracy                           0.90       178
   macro avg       0.91      0.90      0.90       178
weighted avg       0.91      0.90      0.90       178



## COMPROBAMOS EL RESULTADO CON EL CONJUNTO DE TEST

In [11]:

# Configura el modelo con los mejores parámetros
bagging_best = BaggingClassifier(
    estimator=SVC(),  
    n_estimators=10,                     
    max_samples=1.0,                     
    max_features=1.0,                   
    bootstrap=True,
    bootstrap_features=False,
    random_state=0
)

X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Entrena el modelo con los datos de entrenamiento
bagging_best.fit(X_train, y_train)

# Evalúa el modelo en los datos de prueba
y_pred_rfc = bagging_best.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_rfc)
report = classification_report(y_test_final, y_pred_rfc, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[28253   179]
 [    5    44]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      1.00     28432
Fraudulentas       0.20      0.90      0.32        49

    accuracy                           0.99     28481
   macro avg       0.60      0.95      0.66     28481
weighted avg       1.00      0.99      1.00     28481



## GaussianNB

In [13]:
from sklearn.naive_bayes import GaussianNB

X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = GaussianNB()
bagging = BaggingClassifier(estimator=base_estimator, random_state=0)

# Configura el GridSearchCV
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evalúa el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))


Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 1.0, 'max_samples': 0.7, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.899700329637399
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.84      0.98      0.91        89
           1       0.97      0.82      0.89        89

    accuracy                           0.90       178
   macro avg       0.91      0.90      0.90       178
weighted avg       0.91      0.90      0.90       178



## COMPROBAMOS EL RESULTADO CON EL CONJUNTO DE TEST

In [14]:

# Configura el modelo con los mejores parámetros
bagging_best = BaggingClassifier(
    estimator=GaussianNB(),  
    n_estimators=10,                     
    max_samples=0.7,                     
    max_features=1.0,                   
    bootstrap=False,
    bootstrap_features=True,
    random_state=0
)

X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Entrena el modelo con los datos de entrenamiento
bagging_best.fit(X_train, y_train)

# Evalúa el modelo en los datos de prueba
y_pred_rfc = bagging_best.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_rfc)
report = classification_report(y_test_final, y_pred_rfc, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[28390    42]
 [    8    41]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.49      0.84      0.62        49

    accuracy                           1.00     28481
   macro avg       0.75      0.92      0.81     28481
weighted avg       1.00      1.00      1.00     28481



## RandomForestClassifier

In [15]:

from sklearn.ensemble import RandomForestClassifier
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = RandomForestClassifier()
bagging = BaggingClassifier(estimator=base_estimator, random_state=0)

# Configura el GridSearchCV
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evalúa el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))


Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.9265707721506343
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.88      0.97      0.92        89
           1       0.96      0.87      0.91        89

    accuracy                           0.92       178
   macro avg       0.92      0.92      0.92       178
weighted avg       0.92      0.92      0.92       178



## COMPROBAMOS EL RESULTADO CON EL CONJUNTO DE TEST

In [ ]:

# Configura el modelo con los mejores parámetros
bagging_best = BaggingClassifier(
    estimator=RandomForestClassifier(),  
    n_estimators=10,                     
    max_samples=0.7,                     
    max_features=1.0,                   
    bootstrap=False,
    bootstrap_features=True,
    random_state=0
)

X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Entrena el modelo con los datos de entrenamiento
bagging_best.fit(X_train, y_train)

# Evalúa el modelo en los datos de prueba
y_pred_rfc = bagging_best.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_rfc)
report = classification_report(y_test_final, y_pred_rfc, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)